## Type 2 (Historical Method):
- Behavior: A new record is added with the updated information, and the old record is retained with its historical data. Often, metadata like an effective date and expiration date, or a current flag, are used to manage the records.
- Use Case: When it’s important to track historical changes and understand how the dimension has changed over time.

Example: If a customer moves to a new city, a new record with the new city is added, and the old record is kept with the original city.

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SCD Type 2 Example Tables</title>
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        table, th, td {
            border: 1px solid black;
        }
        th, td {
            padding: 10px;
            text-align: center;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>

<h5>Initial Dimension Table:</h5>
<table>
    <tr>
        <th>CustomerID</th>
        <th>CustomerName</th>
        <th>City</th>
        <th>StartDate</th>
        <th>EndDate</th>
        <th>CurrentFlag</th>
    </tr>
    <tr>
        <td>1</td>
        <td>Jane Smith</td>
        <td>New York</td>
        <td>2021-01-01</td>
        <td>NaT</td>
        <td>True</td>
    </tr>
</table>

<h5>Updated Dimension Table with SCD Type 2:</h5>
<table>
    <tr>
        <th>CustomerID</th>
        <th>CustomerName</th>
        <th>City</th>
        <th>StartDate</th>
        <th>EndDate</th>
        <th>CurrentFlag</th>
    </tr>
    <tr>
        <td>1</td>
        <td>Jane Smith</td>
        <td>New York</td>
        <td>2021-01-01</td>
        <td>2022-06-01</td>
        <td>False</td>
    </tr>
    <tr>
        <td>2</td>
        <td>Jane Smith</td>
        <td>Los Angeles</td>
        <td>2022-06-02</td>
        <td>NaT</td>
        <td>True</td>
    </tr>
</table>

</body>
</html>


In [3]:
import pandas as pd
import sqlalchemy as sal

##### Connection to sql server

In [2]:
# removed sql server details for security purpose
connection_string = ('mssql://<sql server _name>\SQLEXPRESS/<database_name>?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
engine = sal.create_engine(connection_string)
conn=engine.connect()

In [ ]:
# Create a target table in sql server name products_dim
    # create table product_dim(
    # product_key int identity(1,1) primary key,
    # product_id int,
    # product_name varchar(100),
    # price int,
    # start_date date,
    # end_date date
    # )

# Source file name products is there with 2 records for the first time
    # product_id,product_name,price
    # 100,Iphone 13,50000
    # 200,HP Laptop Pro,60000

# after first insert the data in target sql table will looks like below: both the records are active
    # product_key	product_id	product_name	price	start_date	end_date
    # 1	100	Iphone 13	50000	2024-01-01	9999-12-31
    # 2	200	HP Laptop Pro	60000	2024-01-01	9999-12-31

# for the second time will get the below data in products source file
    # product_id,product_name,price
    # 100,Iphone 13,60000
    # 300,Iphone 13,70000

# for the thrid time will get the below data in proudcts source file
    # product_id,product_name,price
    # 200,HP Laptop Pro,65000
    # 300,Iphone 13,80000
    # 400,Iphone 14,90000

#

##### Function for to extract the data

In [24]:
def extract():
    products_df = pd.read_csv("products.csv")

    # only retrieving the products which are active from
    products_db_df = pd.read_sql_query("select * from product_dim where end_date='9999-12-31'", conn)
    
    return products_df, products_db_df

#

##### functions to transform the data

In [53]:
def transform(products_df, products_db_df):
    # join the source and target table data based on the product id to identify updated columns
    joined_df = pd.merge(products_df, products_db_df, how='inner', on='product_id')

    # retriving the proudct keys for which we have to update
    update_rows = joined_df['product_key']
    keys = update_rows.to_list()
    product_keys = ','.join([str(key) for key in keys])
    return product_keys
    # update_rows = 
    

#

In [64]:
# function to update the the products target table
def updates(product_keys):
    query = sal.text("update product_dim set end_date= cast(getdate() as date) where product_key in (" + product_keys + ") " )
    p = conn.execute(query)
    conn.commit() 

#

In [67]:
# function to insert the products
def inserts(df_products):
    df_products['start_date'] = pd.to_datetime('now').strftime('%Y-%m-%d')
    df_products['end_date'] = '9999-12-31'
    df_products.to_sql('product_dim', con=conn , index=False , if_exists = 'append')
    conn.commit()

#

In [68]:
products_df, products_db_df = extract()

In [23]:
# for the first time, manually assigning the start date and end date(this will be not be the case in the real time projects)
    # products_df['start_date'] = '2024-01-01'
    # products_df['end_date'] = '9999-12-31'

In [69]:
product_keys = transform(products_df, products_db_df)

In [70]:
# only call updates function if any updated records comes from source
if product_keys != '':
    updates(product_keys)

In [71]:
inserts(products_df)